In [ ]:
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost 
import lightgbm
import catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
%matplotlib 
import shap
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata_ovp=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data_ovp=fdata_ovp.loc[:,[
                     'Relative Atomic Mass',#0
                      'Atomic Number',#1
                      'Ionization Potential',#2
                      'Electronegativity',#3
                      'Number of d electrons',#4
                      'ZIF or MOF Derived',#5
                      'Hard or Soft Templated',#6
                      'Graphene/Carbon Nanosheets or other 2D Structures',#7
                      'Polymer Derived',#8
                      'Carbon Nanofiber/Nanotubes',#9
                      'Carbon Black Derived',#10
                      'Biomass or other Organic Derived',#11  
                      'Main Transition Metal Content (wt. %)',#12
                      'Nitrogen Cotent (wt. %)',#13
                      'Metal-N Coordination Number (XAS)',#14    
                      'Pyridinic N Ratio',#15
                      'Pyrrolic N Ratio',#16
                      'Graphitic N Ratio',#17
                      'Oxidized N Ratio',#18
                      'Raman ID/IG Ratio',#19
                      'BET Surface Area (m2/g)',#20
                      'Acid Wash',#21
                      'Pyrolysis Temperature (°C)',#22
                      'Pyrolysis Time (h)',#23
                      'Rising Rate (°C min-1)',#24
                      'Flow Cell/H-type Cell',#25
                      'Electrolyte Concentration (M)',#26
                      'Catalyst Loading (mg cm-2)',#27
                      'Nafion Membrane Thickness (μm)',#28
                      'Carbon Paper/Glassy Carbon',#29
                      'Electrode Area cm2',#30
                      'Electrolyte pH',#31
                      'Overpotential (V)'#32
                        ]]



###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input_ovp=raw_data_ovp.iloc[:,0:32]
raw_output_ovp=raw_data_ovp.iloc[:,32]
# raw_input_global=raw_data.iloc[:,0:32]
# raw_output_global=raw_data.iloc[:,32]
X_ovp=raw_input_ovp.values.astype(np.float32)
y_ovp=raw_output_ovp.values.astype(np.float32)
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})



In [ ]:
def gridsearch(model,param,algorithm_name,X_train, X_test, y_train, y_test):
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=2)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test.values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    
    result_train = best_model.predict(X_train)
    x_prediction_07_train=result_train
    y_real_07_train=y_train.values
    x_prediction_07_series_train=pd.Series(x_prediction_07_train)
    y_real_07_series_train=pd.Series(y_real_07_train)
    
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    
    corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
    error_val_train= compute_mae_mse_rmse(x_prediction_07_train,y_real_07_train)
    
    print(algorithm_name)
    print(best_model.feature_importances_)
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val,'TEST R2',error_val[3],'TEST CORR',corr_ann)
    print(error_val_train,'TRAIN R2',error_val_train[3],'TRAIN CORR',corr_ann_train)
#     x_y_x=np.arange(0.2,1.1,0.1)
#     x_y_y=np.arange(0.2,1.1,0.1)
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name+' Test Set',alpha=0.75)
#     ax.scatter(x_prediction_07_train,y_real_07_train,color='blue',label=algorithm_name+' Training Set',alpha=0.25,marker="^")
#     ax.plot(x_y_x,x_y_y)
#     plt.legend()
#     plt.xlabel(u"Predicted_Overpotential_@_Maximum_Faradic_Efficiency")
#     plt.ylabel(u"Real_Overpotential_@_Maximum_Faradic_Efficiency")
#     print('finished')
    return best_model

In [ ]:
seed_ovp=1384
X_train_ovp, X_test_ovp, y_train_ovp, y_test_ovp = train_test_split(raw_input_ovp, raw_output_ovp, test_size=.1,random_state=seed_ovp)

In [ ]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbose=0)
param_xg = {
 'booster':['gbtree'],
 'learning_rate':[0.05],
 'n_estimators':[100],
 'max_depth':[11],
 'subsample':[0.55],
 'reg_alpha':[0.01],
 'reg_lambda':[1e-05]
}
XG_OVP=gridsearch(model_XGBRegressor,param_xg,'XGBoost',X_train_ovp, X_test_ovp, y_train_ovp, y_test_ovp)

In [ ]:
###########loading data##########
fdata_TOF=pd.read_csv('database_filled_TOF_MCL.csv',encoding="gbk")
raw_data_TOF=fdata_TOF.loc[:,[
                     'Relative Atomic Mass',#0
                      'Atomic Number',#1
                      'Ionization Potential',#2
                      'Electronegativity',#3
                      'Number of d electrons',#4
                      'ZIF or MOF Derived',#5
                      'Hard or Soft Templated',#6
                      'Graphene/Carbon Nanosheets or other 2D Structures',#7
                      'Polymer Derived',#8
                      'Carbon Nanofiber/Nanotubes',#9
                      'Carbon Black Derived',#10
                      'Biomass or other Organic Derived',#11  
                      'Main Transition Metal Content (wt. %)',#12
                      'Nitrogen Cotent (wt. %)',#13
                      'Metal-N Coordination Number (XAS)',#14    
                      'Pyridinic N Ratio',#15
                      'Pyrrolic N Ratio',#16
                      'Graphitic N Ratio',#17
                      'Oxidized N Ratio',#18
                      'Raman ID/IG Ratio',#19
                      'BET Surface Area (m2/g)',#20
                      'Acid Wash',#21
                      'Pyrolysis Temperature (°C)',#22
                      'Pyrolysis Time (h)',#23
                      'Rising Rate (°C min-1)',#24
                      'Flow Cell/H-type Cell',#25
                      'Electrolyte Concentration (M)',#26
                      'Catalyst Loading (mg cm-2)',#27
                      'Nafion Membrane Thickness (μm)',#28
                      'Carbon Paper/Glassy Carbon',#29
                      'Electrode Area cm2',#30
                      'Electrolyte pH',#31
                      'TOF@Maximum FE(h-1) (log)'#32
                        ]]



###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input_TOF=raw_data_TOF.iloc[:,0:32]
raw_output_TOF=raw_data_TOF.iloc[:,32]

In [ ]:
seed_TOF=9
X_train_TOF, X_test_TOF, y_train_TOF, y_test_TOF = train_test_split(raw_input_TOF, raw_output_TOF, test_size=.1,random_state=seed_TOF)

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
 'learning_rate':[0.1],
 'n_estimators':[400],
 'max_depth':[5],
 'subsample':[1],
    'reg_lambda':[0]
}
CAT_TOF=gridsearch(model_CatRegressor,param_cat,'CatBoost',X_train_TOF, X_test_TOF, y_train_TOF, y_test_TOF)

In [ ]:
Sc=[44.956,21,6.57,1.36,1]
Ti=[47.867,22,6.81,1.54,2]
V=[50.942,23,6.76,1.63,3]
Cr=[51.996,24,6.74,1.66,4]
Mn=[54.938,25,7.4,1.56,5]
Fe=[55.845,26,7.83,1.83,6]
Co=[58.933,27,7.81,1.88,7]
Ni=[58.6934,28,7.61,1.91,8]
Cu=[63.546,29,7.69,1.9,10]
Zn=[65.38,30,9.35,1.65,10]
Ga=[69.723,31,5.97,1.81,10]
Y=[88.905,39,6.5,1.22,1]
Zr=[91.224,40,6,1.33,2]
Nb=[92.906,41,6.76,1.6,4]
Mo=[95.94,42,7.35,2.16,5]
Tc=[98.906,43,7.28,1.9,5]
Ru=[101.07,44,7.5,2.2,7]
Rh=[102.905,45,7.7,2.28,8]
Pd=[106.42,46,8.3,2.2,10]
Ag=[107.868,47,7.54,1.93,10]
Cd=[112.411,48,8.95,1.69,10]
In=[114.818,49,8.95,1.78,10]
Sn=[118.71,50,7.37,1.96,10]
La=[138.905,57,5.5,1.1,1]
Ce=[140.116,58,6.91,1.12,1]
Pr=[140.904,59,5.76,1.13,10]
Nd=[144.242,60,6.31,1.14,10]
Pm=[144.912,61,5.55,1.13,10]
Sm=[150.36,62,6.55,1.1,10]
Eu=[151.964,63,5.67,1.2,10]
Gd=[157.25,64,6.65,1.2,1]
Tb=[158.925,65,6.74,1.2,10]
Dy=[162.5,66,6.82,1.22,10]
Ho=[164.93,67,6.022,1.23,10]
Er=[167.529,68,6.108,1.23,10]
Tm=[168.934,69,6.184,1.25,10]
Yb=[173.04,70,7.06,1.1,10]
Lu=[174.967,71,5.4259,1.27,1]
Hf=[178.49,72,6.8251,1.3,2]
Ta=[180.947,73,7.89,1.5,3]
W=[183.84,74,7.98,2.36,4]
Re=[186.207,75,7.88,1.9,5]
Os=[190.23,76,8.7,2.2,6]
Ir=[192.217,77,9.1,2.2,7]
Pt=[195.084,78,8.9,2.28,9]
Au=[196.966,79,9.19,2.54,10]
Hg=[200.59,80,10.39,2,10]
Tl=[204.383,81,6.08,1.62,10]
Pb=[207.2,82,7.38,2.33,10]
Bi=[208.98,83,7.25,2.02,10]

In [ ]:
ZIF=[1,0,0,0,0,0,0]
TEMPLATE=[0,1,0,0,0,0,0]
GRAPHENE=[0,0,1,0,0,0,0]
POLYMER=[0,0,0,1,0,0,0]
NANOTUBE=[0,0,0,0,1,0,0]
BLACK=[0,0,0,0,0,1,0]
BIO=[0,0,0,0,0,0,1]

In [ ]:
OVP_DICT={}
TOF_DICT={}
MIX_DICT={}

In [ ]:

for element_type in [Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,La,Ce,Pr,Nd,Pm,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi]:
    for precursor_type in [ZIF,TEMPLATE,GRAPHENE,POLYMER,NANOTUBE,BLACK,BIO]:
        for MT in np.arange(0.1,5.1,0.5):
            for NC in np.arange(1,8,0.5):
                for MNN in [3,4,5]:
                    for PYRID in np.arange(0,80,10):
                        for PYRRO in np.arange(0,25,5):
                            for TEMP in range(800,1100,100):
                                GRA=95-PYRID-PYRRO
                                if GRA<0:
                                    GRA=0
                                test_variable=element_type+precursor_type+[MT,NC,MNN,PYRID,PYRRO,GRA,5,1.2,800,0,TEMP,2,5,0,0.1,1,177.8,1,1,6.8]
                                variable=np.array([test_variable])
                                OVP=XG_OVP.predict(variable)[0]
                                TOF=CAT_TOF.predict(variable)[0]
                                MIX_DIVIDE=OVP/TOF
                                OVP_DICT[tuple(test_variable)]=OVP
                                TOF_DICT[tuple(test_variable)]=TOF
                                MIX_DICT[tuple(test_variable)]=MIX_DIVIDE

                                                    

In [ ]:
min_OVP=min(zip(OVP_DICT.values(),OVP_DICT.keys()))
print(min_OVP)

In [ ]:
max_TOF=max(zip(TOF_DICT.values(),TOF_DICT.keys()))
print(max_TOF)

In [ ]:
min_MIX=min(zip(MIX_DICT.values(),MIX_DICT.keys()))
print(min_MIX)

In [ ]:
TEST2=sorted(MIX_DICT,key=MIX_DICT.get, reverse=True)

In [ ]:
AAB={}
ABB={}

In [ ]:
for each in OVP_DICT:
    TEST_AAB=(OVP_DICT[each]*OVP_DICT[each])/TOF_DICT[each]
    TEST_ABB=OVP_DICT[each]/(TOF_DICT[each]*TOF_DICT[each])
    AAB[each]=TEST_AAB
    ABB[each]=TEST_ABB

In [ ]:
TEST_AAB=sorted(AAB,key=AAB.get, reverse=True)
TEST_ABB=sorted(ABB,key=ABB.get, reverse=True)

In [ ]:
for i in range (1,21):
    each=list(TEST_AAB[-i])
    variable=np.array([each])
    OVP=XG_OVP.predict(variable)[0]
    TOF=CAT_TOF.predict(variable)[0]
    print(OVP,TOF)

In [ ]:
for i in range (1,21):
    each=list(TEST_ABB[-i])
    variable=np.array([each])
    OVP=XG_OVP.predict(variable)[0]
    TOF=CAT_TOF.predict(variable)[0]
    print(OVP,TOF)

In [ ]:
for i in range (1,21):
    each=list(TEST2[-i])
    variable=np.array([each])
    OVP=XG_OVP.predict(variable)[0]
    TOF=CAT_TOF.predict(variable)[0]
    print(OVP,TOF)